## Working with DataFrames

### Data Import
Lets import the same data sets

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display

In [3]:
# pass in column names for each CSV (info in README file)
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
url_user= 'https://github.com/ShankarRamakrishnan/PandasTutorial/raw/master/data/ml-100k/u.user'
users = pd.read_csv(url_user, sep='|', names=u_cols)

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
url_data = 'https://github.com/ShankarRamakrishnan/PandasTutorial/blob/master/data/ml-100k/u.data?raw=true'
ratings = pd.read_csv(url_data, sep='\t', names=r_cols)

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
url_item = 'https://github.com/ShankarRamakrishnan/PandasTutorial/raw/master/data/ml-100k/u.item'
movies = pd.read_csv(url_item, sep='|', names=m_cols, usecols=range(5), encoding = "ISO-8859-1")

### Joining

Like SQL's **JOIN** clause, `pandas.merge` allows two DataFrames to be joined on one or more keys. The function provides a series of parameters `(on, left_on, right_on, left_index, right_index)` allowing you to specify the columns or indexes on which to join.

By default, `pandas.merge` operates as an *inner join*, which can be changed using the `how` parameter.

From the function's docstring:

> how : {'left', 'right', 'outer', 'inner'}, default 'inner'

>    * left: use only keys from left frame (SQL: left outer join)

>    * right: use only keys from right frame (SQL: right outer join)

>    * outer: use union of keys from both frames (SQL: full outer join)

>    * inner: use intersection of keys from both frames (SQL: inner join)

Below are some examples of what each look like.

Our MovieLens data is a good example of this - a rating requires both a user and a movie, and the datasets are linked together by a key - in this case, the user_id and movie_id. It's possible for a user to be associated with zero or many ratings and movies. Likewise, a movie can be rated zero or many times, by a number of different users.

In [11]:
left_frame = pd.DataFrame({'key': range(5), 
                           'left_value': ['a', 'b', 'c', 'd', 'e']})
right_frame = pd.DataFrame({'key': range(2, 7), 
                           'right_value': ['f', 'g', 'h', 'i', 'j']})
print('left frame')
display(left_frame)
print('right frame')
display(right_frame)

left frame


,key,left_value
0,0,a
1,1,b
2,2,c
3,3,d
4,4,e


right frame


,key,right_value
0,2,f
1,3,g
2,4,h
3,5,i
4,6,j


**inner join (default)**

In [12]:
pd.merge(left_frame, right_frame, on='key', how='inner')

,key,left_value,right_value
0,2,c,f
1,3,d,g
2,4,e,h


We lose values from both frames since certain keys do not match up.  The SQL equivalent is:

```
    SELECT left_frame.key, left_frame.left_value, right_frame.right_value
    FROM left_frame
    INNER JOIN right_frame
        ON left_frame.key = right_frame.key;
```

Had our *key* columns not been named the same, we could have used the *left_on* and *right_on* parameters to specify which fields to join from each frame.
```python
    pd.merge(left_frame, right_frame, left_on='left_key', right_on='right_key')
```
Alternatively, if our keys were indexes, we could use the `left_index` or `right_index` parameters, which accept a True/False value. You can mix and match columns and indexes like so:
```python
    pd.merge(left_frame, right_frame, left_on='key', right_index=True)
```

In [26]:
display(movies.head(5))
ratings.head(5)

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [27]:
pd.merge(movies, ratings, on='movie_id', how='inner')

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,unix_timestamp
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532
1,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,287,5,875334088
2,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,148,4,877019411
3,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,280,4,891700426
4,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,66,3,883601324
...,...,...,...,...,...,...,...,...
99995,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,863,1,889289570
99996,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,863,3,889289491
99997,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),863,2,889289570
99998,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,896,3,887160722


**left outer join**

In [13]:
pd.merge(left_frame, right_frame, on='key', how='left')

,key,left_value,right_value
0,0,a,NaN
1,1,b,NaN
2,2,c,f
3,3,d,g
4,4,e,h


We keep everything from the left frame, pulling in the value from the right frame where the keys match up. The right_value is NULL where keys do not match (NaN).

SQL Equivalent:

    SELECT left_frame.key, left_frame.left_value, right_frame.right_value
    FROM left_frame
    LEFT JOIN right_frame
        ON left_frame.key = right_frame.key;

In [30]:
display(users.head(5))
ratings.head(5)

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [32]:
display('users count', len(users.index))
display('ratings count', len(ratings.index))

'users count'

943

'ratings count'

100000

In [33]:
pd.merge(users, ratings, on='user_id', how='left')

,user_id,age,sex,occupation,zip_code,movie_id,rating,unix_timestamp
0,1,24,M,technician,85711,61,4,878542420
1,1,24,M,technician,85711,189,3,888732928
2,1,24,M,technician,85711,33,4,878542699
3,1,24,M,technician,85711,160,4,875072547
4,1,24,M,technician,85711,20,4,887431883
...,...,...,...,...,...,...,...,...
99995,943,22,M,student,77841,415,1,888640027
99996,943,22,M,student,77841,219,4,888639575
99997,943,22,M,student,77841,796,3,888640311
99998,943,22,M,student,77841,739,4,888639929


**right outer join**

In [34]:
pd.merge(left_frame, right_frame, on='key', how='right')

,key,left_value,right_value
0,2,c,f
1,3,d,g
2,4,e,h
3,5,NaN,i
4,6,NaN,j


This time we've kept everything from the right frame with the left_value being NULL where the right frame's key did not find a match.

SQL Equivalent:

    SELECT right_frame.key, left_frame.left_value, right_frame.right_value
    FROM left_frame
    RIGHT JOIN right_frame
        ON left_frame.key = right_frame.key;

**full outer join**

In [35]:
pd.merge(left_frame, right_frame, on='key', how='outer')

,key,left_value,right_value
0,0,a,NaN
1,1,b,NaN
2,2,c,f
3,3,d,g
4,4,e,h
5,5,NaN,i
6,6,NaN,j


We've kept everything from both frames, regardless of whether or not there was a match on both sides. Where there was not a match, the values corresponding to that key are NULL.

SQL Equivalent (though some databases don't allow FULL JOINs (e.g. MySQL)):

    SELECT IFNULL(left_frame.key, right_frame.key) key
            , left_frame.left_value, right_frame.right_value
    FROM left_frame
    FULL OUTER JOIN right_frame
        ON left_frame.key = right_frame.key;

### Combining

pandas also provides a way to combine DataFrames along an axis - `pandas.concat`. While the function is equivalent to SQL's UNION clause, there's a lot more that can be done with it.

`pandas.concat` takes a list of Series or DataFrames and returns a Series or DataFrame of the concatenated objects. Note that because the function takes list, you can combine many objects at once.

In [36]:
pd.concat([left_frame, right_frame])

,key,left_value,right_value
0,0,a,NaN
1,1,b,NaN
2,2,c,NaN
3,3,d,NaN
4,4,e,NaN
0,2,NaN,f
1,3,NaN,g
2,4,NaN,h
3,5,NaN,i
4,6,NaN,j


By default, the function will vertically append the objects to one another, combining columns with the same name. We can see above that values not matching up will be NULL.

Additionally, objects can be concatentated side-by-side using the function's *axis* parameter.

In [37]:
pd.concat([left_frame, right_frame], axis=1)

,key,left_value,key,right_value
0,0,a,2,f
1,1,b,3,g
2,2,c,4,h
3,3,d,5,i
4,4,e,6,j


`pandas.concat` can be used in a variety of ways; however, I've typically only used it to combine Series/DataFrames into one unified object. The [documentation](http://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-objects) has some examples on the ways it can be used.

### Grouping

pandas `groupby` method comes largely from the [split-apply-combine strategy for data analysis](http://www.jstatsoft.org/v40/i01/paper).  

When approaching a data analysis problem, you'll often break it apart into manageable pieces, perform some operations on each of the pieces, and then put everything back together again (this is the gist split-apply-combine strategy). pandas `groupby` is great for these problems (R users should check out the [plyr](http://plyr.had.co.nz/) and [dplyr](https://github.com/hadley/dplyr) packages).

Assume we have a DataFrame and want to get the average for each group - visually, the split-apply-combine method looks like this:

![Source: Gratuitously borrowed from [Hadley Wickham's Data Science in R slides](http://courses.had.co.nz/12-oscon/)](http://i.imgur.com/yjNkiwL.png)

Loading chicago salary data. Since the data contains a dollar sign for each salary, python will treat the field as a series of strings. We can use the `converters` parameter to change this when reading in the file.

>converters : dict. optional

>* Dict of functions for converting values in certain columns. Keys can either be integers or column labels

In [38]:
url = 'https://github.com/ShankarRamakrishnan/PandasTutorial/blob/master/data/city-of-chicago-salaries.csv?raw=true'
headers = ['name', 'title', 'department', 'salary']
chicago = pd.read_csv(url,
                      header=0,
                      names=headers,
                      converters={'salary': lambda x: float(x.replace('$', ''))})
chicago.head()

,name,title,department,salary
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,85512.0
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,75372.0
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,80916.0
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,99648.0
4,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,89440.0


pandas `groupby` returns a DataFrameGroupBy object which has a variety of methods, many of which are similar to standard SQL aggregate functions.

In [39]:
by_dept = chicago.groupby('department')
by_dept

In [ ]:
by_dept.groups

Calling `count` returns the total number of NOT NULL values within each column. If we were interested in the total number of records in each group, we could use `size`.

In [40]:
display(by_dept.count().head()) # NOT NULL records within each column
display(by_dept.size().head()) # total records for each department

,name,title,salary
department,,,
ADMIN HEARNG,42,42,42
ANIMAL CONTRL,61,61,61
AVIATION,1218,1218,1218
BOARD OF ELECTION,110,110,110
BOARD OF ETHICS,9,9,9


department
ADMIN HEARNG           42
ANIMAL CONTRL          61
AVIATION             1218
BOARD OF ELECTION     110
BOARD OF ETHICS         9
dtype: int64

Summation can be done via `sum`, averaging by `mean`, etc. (if it's a SQL function, chances are it exists in pandas). 

And there's median too, something not available in most databases.


In [45]:
display(by_dept.sum().head()) # total salaries of each department
display(by_dept.mean().head()) # average salary of each department
display(by_dept.median().head()) # middle value or average of middle values

,salary
department,
ADMIN HEARNG,2954148.00
ANIMAL CONTRL,3484408.80
AVIATION,86037387.44
BOARD OF ELECTION,6130176.00
BOARD OF ETHICS,734856.00


,salary
department,
ADMIN HEARNG,70336.857143
ANIMAL CONTRL,57121.455738
AVIATION,70638.249130
BOARD OF ELECTION,55728.872727
BOARD OF ETHICS,81650.666667


,salary
department,
ADMIN HEARNG,63456.0
ANIMAL CONTRL,55212.0
AVIATION,70408.0
BOARD OF ELECTION,49236.0
BOARD OF ETHICS,75240.0


Operations can also be done on an individual Series within a grouped object. Say we were curious about the five departments with the most distinct titles - the pandas equivalent to:

    SELECT department, COUNT(DISTINCT title)
    FROM chicago
    GROUP BY department
    ORDER BY 2 DESC
    LIMIT 5;

pandas is a lot less verbose here ...

In [46]:
by_dept.title.nunique().sort_values(ascending=False)[:5]

department
WATER MGMNT    153
TRANSPORTN     150
POLICE         130
AVIATION       125
HEALTH         118
Name: title, dtype: int64